In [2]:
import sqlite3

conn = sqlite3.connect('../database/database.db')

cursor = conn.cursor()

In [10]:
import sqlite3
from IPython.display import Image

connect = sqlite3.connect('ass3.db')
cursor = connect.cursor()


def query(input):
    cursor.execute(input)

    records = cursor.fetchall()

    for record in records:
        print(record)


In [14]:
query('''
      select *
      from OfficerDim
      ''')

OperationalError: no such table: OfficerDim